In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Activation, RepeatVector, Dropout, Dense
from keras.layers import LSTM
from keras.layers.wrappers import TimeDistributed
import numpy as np
import os 

# Suppress the TensorFlow warning
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Function to convert integer input to vectors for RNN using one hot encoding

def encode(X,seq_len, vocab_size):
    x = np.zeros((len(X),seq_len, vocab_size), dtype=np.float32)
    for ind,batch in enumerate(X):
        for j, elem in enumerate(batch):
            x[ind, j, elem] = 1
    return x

In [3]:
# Generator function to generate infinite-stream of inputs for training

def batch_gen(batch_size=32, seq_len=10, max_no=100):
    # Randomly generate a batch of integer sequences (X) and its sorted counterpart (Y)
    x = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)
    y = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)

    while True:
	# Generates a batch of input
        X = np.random.randint(max_no, size=(batch_size, seq_len))

        Y = np.sort(X, axis=1)

        for ind,batch in enumerate(X):
            for j, elem in enumerate(batch):
                x[ind, j, elem] = 1

        for ind,batch in enumerate(Y):
            for j, elem in enumerate(batch):
                y[ind, j, elem] = 1

        yield x, y
        x.fill(0.0)
        y.fill(0.0)

In [4]:
def create_model(seq_len, max_no, n_layers, hidden_size):
    model = Sequential()
    # the encoder LSTM
    model.add(LSTM(hidden_size, input_shape=(seq_len, max_no)))
    # in next layer, repeat the input seq_len times
    model.add(RepeatVector(seq_len))
    # decoder RNN, which will return output sequence
    for _ in range(n_layers):
        model.add(LSTM(hidden_size, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(max_no)))
    model.add(Activation('softmax'))
    return model

In [5]:
# Global parameters.
batch_size=32
seq_len = 15 # number of elements in sequence to sort
max_no = 100 # upper range of the numbers in sequence
hidden_size = 128
n_layers = 2
epochs = 100000

model = create_model(seq_len, max_no, n_layers, hidden_size)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
# Now the training loop, we'll sample input batches from the generator function written previously and feed it to the RNN for learning

for ind,(X,Y) in enumerate(batch_gen(batch_size, seq_len, max_no)):
    loss, acc = model.train_on_batch(X, Y)
    # We'll test RNN after each 500 iteration to check how well it is performing
    if ind % 500 == 0:
        testX = np.random.randint(max_no, size=(1, seq_len))
        test = encode(testX, seq_len, max_no)
        #print(testX, ': input sequence')
        print(ind, ': epochs')
        print(acc, ': accuracy')
        loss_str = '{:4.3}'.format(loss)
        acc_str = '{:4.3}'.format(acc)
        
        y = model.predict(test, batch_size=1)
        np_sorted = np.sort(testX)[0]
        rnn_sorted = np.argmax(y, axis=2)[0]
        is_equal = np.array_equal(np_sorted, rnn_sorted)
        print(np_sorted, ': sorted by NumPy algorithm')
        print(rnn_sorted, ': sorted by trained RNN')
        print("\n")
        
        if acc > 0.98 :
            model.save('model.h5')
            break
        
    if ind > epochs:
        # Save trained model
        model.save('model.h5')
        break
    

(0, ': epochs')
(0.008333334, ': accuracy')
(array([ 7,  9, 12, 13, 15, 16, 20, 24, 26, 37, 51, 67, 71, 72, 76]), ': sorted by NumPy algorithm')
(array([57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57]), ': sorted by trained RNN')


(500, ': epochs')
(0.09166667, ': accuracy')
(array([12, 24, 34, 38, 48, 49, 51, 54, 56, 62, 62, 68, 71, 79, 85]), ': sorted by NumPy algorithm')
(array([ 1, 19, 34, 38, 44, 50, 51, 56, 59, 63, 69, 69, 76, 80, 90]), ': sorted by trained RNN')


(1000, ': epochs')
(0.12708333, ': accuracy')
(array([29, 40, 44, 50, 51, 54, 67, 75, 81, 85, 91, 92, 98, 98, 99]), ': sorted by NumPy algorithm')
(array([23, 37, 44, 46, 53, 58, 63, 73, 79, 82, 89, 92, 99, 99, 99]), ': sorted by trained RNN')


(1500, ': epochs')
(0.17083333, ': accuracy')
(array([ 5,  8,  8, 14, 16, 38, 47, 48, 64, 68, 78, 82, 86, 88, 95]), ': sorted by NumPy algorithm')
(array([ 0,  4,  8, 11, 19, 39, 46, 52, 60, 68, 78, 82, 85, 89, 93]), ': sorted by trained RNN')


(2000, ': epochs')


(17000, ': epochs')
(0.78541666, ': accuracy')
(array([12, 16, 18, 22, 30, 37, 46, 53, 54, 56, 58, 66, 70, 86, 95]), ': sorted by NumPy algorithm')
(array([12, 16, 18, 22, 30, 38, 46, 53, 55, 56, 58, 66, 70, 86, 95]), ': sorted by trained RNN')


(17500, ': epochs')
(0.7583333, ': accuracy')
(array([ 1, 13, 18, 20, 24, 39, 43, 51, 62, 64, 74, 74, 75, 85, 91]), ': sorted by NumPy algorithm')
(array([ 1, 13, 18, 20, 25, 39, 43, 51, 63, 64, 74, 74, 75, 85, 91]), ': sorted by trained RNN')


(18000, ': epochs')
(0.76666665, ': accuracy')
(array([ 6,  7, 12, 32, 36, 62, 64, 75, 76, 77, 83, 89, 91, 96, 97]), ': sorted by NumPy algorithm')
(array([ 6,  7, 12, 32, 36, 62, 64, 75, 76, 77, 83, 89, 91, 96, 97]), ': sorted by trained RNN')


(18500, ': epochs')
(0.7708333, ': accuracy')
(array([ 3,  4,  7, 14, 36, 41, 50, 55, 56, 60, 61, 66, 71, 87, 88]), ': sorted by NumPy algorithm')
(array([ 3,  4,  7, 14, 36, 41, 50, 55, 56, 60, 61, 66, 71, 87, 88]), ': sorted by trained RNN')


(19000, ': epo

(34000, ': epochs')
(0.95, ': accuracy')
(array([ 4, 18, 19, 29, 44, 53, 53, 53, 57, 59, 60, 63, 64, 80, 94]), ': sorted by NumPy algorithm')
(array([ 4, 18, 19, 29, 44, 53, 53, 53, 57, 59, 60, 63, 64, 80, 94]), ': sorted by trained RNN')


(34500, ': epochs')
(0.95208335, ': accuracy')
(array([ 8, 11, 28, 35, 53, 60, 60, 60, 64, 73, 73, 75, 77, 91, 97]), ': sorted by NumPy algorithm')
(array([ 8, 11, 28, 35, 53, 60, 60, 60, 64, 73, 73, 75, 77, 91, 97]), ': sorted by trained RNN')


(35000, ': epochs')
(0.9604167, ': accuracy')
(array([ 0,  1,  7, 13, 19, 28, 28, 31, 55, 60, 61, 64, 69, 76, 81]), ': sorted by NumPy algorithm')
(array([ 0,  1,  7, 13, 19, 28, 28, 31, 55, 60, 61, 64, 69, 76, 81]), ': sorted by trained RNN')


(35500, ': epochs')
(0.9770833, ': accuracy')
(array([ 1,  6, 15, 19, 19, 21, 25, 42, 48, 62, 64, 73, 76, 92, 96]), ': sorted by NumPy algorithm')
(array([ 1,  6, 15, 19, 19, 21, 25, 42, 48, 62, 64, 73, 76, 92, 96]), ': sorted by trained RNN')


(36000, ': epochs')
